<a href="https://colab.research.google.com/github/m0o0scar/nerfstudio/blob/main/colab/NeRFStudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/en/latest/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/en/latest/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/en/latest/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


[![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)](https://github.com/nerfstudio-project/nerfstudio)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

# Setup Environment

In [ ]:
# @title ⬇️ Install dependencies
# If the installation fail, please visit this notebook to rebuild the tinycudann & nerfstudio
# https://colab.research.google.com/drive/17px0qexefKh1O5IfKijT86rC2XAHYxiv?usp=sharing
%%time
%cd /content

# /content/gdrive/MyDrive/colab/prebuilt
#   - 20230313-014335
#   - 20230305-081243
#   - latest
#     - tinycudann_and_nerfstudio
#       - nerfstudio-0.1.19-py3-none-any.whl
#       - tinycudann-1.7-cp39-cp39-linux_x86_64.whl
#       - ...
#     - colmap
#       - local.zip
wheelhouse = '/content/gdrive/MyDrive/colab/prebuilt/latest'

# Mount Google Drive
import os
path = "/content"
os.chdir(path)
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# nerfstudio won't install from source without upgrading pip
!pip install --upgrade pip

# TinyCuda
!pip install --no-index --find-links=$wheelhouse/tinycudann_and_nerfstudio tinycudann 

# nerfstudio
!pip install --no-index --find-links=$wheelhouse/tinycudann_and_nerfstudio nerfstudio

# COLMAP
%cd /content/
!sudo apt-get install \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgflags-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev \
    nvidia-cuda-toolkit \
    nvidia-cuda-toolkit-gcc
!cp $wheelhouse/colmap/local.zip .
!unzip local.zip -d /usr/
!chmod +x /usr/local/bin/colmap

# download predefined camera paths
!rm -rf predefined_camera_paths
!gdown "1-8WkBrf3Rb3Ocq3ML6dPj46azORiJA8t&confirm=t"
!unzip predefined_camera_paths.zip
!rm -rf predefined_camera_paths.zip __MACOSX

# localtunnel for the viewer
!npm i -g localtunnel

/content
Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in links: /content/gdrive/MyDrive/colab/prebuilt/latest/tinycudann_and_nerfstudio
Processing ./gdrive/MyDrive/colab/prebuilt/20230313-014335/tinycudann_and_nerfstudio/tinycudann-1.7-cp39-cp39-linux_x86_64.whl
Looking in links: /content/gdrive/MyDrive/colab/prebuilt/latest/tinycudann_and_nerfstudio
Processing ./gdrive/MyDrive/colab/prebuilt/20230313-014335/tinycudann_and_nerfstudio/nerfstudio-0.1.19-py3-none-any.whl
Processing ./gdrive/MyDrive/colab/prebuilt/20230313-014335/tinycudann_and_nerfstudio/xatlas-0.0.7-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing ./gdrive/MyDrive/colab/prebuilt/20230313-014335/ti

In [ ]:
# @title ⚙️ Set project parameters

# @markdown > Choose working directory
folder = '/content/gdrive/MyDrive/time-machine/nerfstudio' #@param {type: "string"}
project = 'looking-glass' #@param {type: "string"}
root = folder + '/' + project
!mkdir -p $root

# @markdown > nerfstudio will create a folder in the following path for storing training checkpoint and stuff:<br/>
# @markdown > `<folder>/<project>/model/nerfacto/<experiment_name>`
experiment_name = 'df-4' #@param {type: "string"}
downscale_factor = 4 #@param {type: "number"}

# # @markdown > If the input is a video, or images extracted from a video, check this to speed up the data processing.
# is_sequential = True #@param {type: 'boolean'}

# Pre-processing

In [ ]:
# @title 🧽 Process raw images / video
%%time
import glob
import os

%cd $root

output_dir = root + '/processed'

if os.path.exists(root + '/processed/transforms.json'):
  print('Data already processed')
else:
  # Are there any raw images in root folder?
  if os.path.exists(root + '/raw_images'):
    raw_images = root + '/raw_images'
    # matching_method = 'sequential' if is_sequential else 'exhaustive'
    matching_method = 'exhaustive'
    print(f'Processing raw images with method {matching_method} at {raw_images} ...')
    !ns-process-data images --data $raw_images --output-dir $output_dir --matching-method $matching_method
    print('Done')

  # Is there any video in root folder?
  else:
    video_paths = []
    for ext in ('*.mp4', '*.MP4', '*.mov', '*.MOV'):
      video_paths.extend(glob.glob(root + '/' + ext))

    if len(video_paths) > 0:
      video_path = video_paths[0]
      print(f'Processing video file at {video_path} ...')
      !ns-process-data video --data $video_path --output-dir $output_dir --matching-method sequential
      print('Done')
    else:
      print('Could not find supported input file(s)')

# For debugging
# cd /content/gdrive/MyDrive/time-machine/nerfstudio/looking-glass-fps-0.5
# rm -rf processed
# ns-process-data images --data raw_images --output-dir processed --matching-method sequential
# colmap feature_extractor --database_path processed/colmap/database.db --image_path processed/images --ImageReader.single_camera 1 --ImageReader.camera_model OPENCV --SiftExtraction.use_gpu 1

/content/gdrive/MyDrive/time-machine/nerfstudio/looking-glass
Processing raw images with method exhaustive at /content/gdrive/MyDrive/time-machine/nerfstudio/looking-glass/raw_images ...
(    ● ) Copying images...[03:17:14] 🎉 Done copying images.                                                             ]8;id=541433;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=978194;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/process_data/process_data_utils.py#216\216]8;;\
(     ●) Copying images...
▃ Downscaling images...
[03:17:59] 🎉 Done downscaling images.                                                         ]8;id=795440;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=167798;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/process_data/process_data_utils.py#337\337]8;;\
🌘  Running COLMAP feature extrac

# Training

In [ ]:
# @title 🏃‍♀️ Start NeRF training
%%time
!ns-train nerfacto \
  --output-dir $root \
  --experiment-name model \
  --timestamp $experiment_name \
  --viewer.websocket-port 7007 \
  nerfstudio-data \
  --data $root/processed \
  --downscale-factor $downscale_factor

# TODO: resume from checkpoint
# --load-dir $root/model/nerfacto/$experiment_name/nerfstudio_models

# Note: for debugging in termial
# cd /content/gdrive/MyDrive/time-machine/nerfstudio/looking-glass-fps-0.5
# ns-train nerfacto --output-dir . --experiment-name model --timestamp 0 --viewer.websocket-port 7007 nerfstudio-data --data processed --downscale-factor 1

Streaming output truncated to the last 5000 lines.
26420 (88.07%)      47.637 ms            2 m, 50 s            88.52 K                                
26430 (88.10%)      45.806 ms            2 m, 43 s            91.09 K                                
26440 (88.13%)      49.093 ms            2 m, 54 s            86.12 K                                
26450 (88.17%)      50.526 ms            2 m, 59 s            83.77 K                                
26460 (88.20%)      49.351 ms            2 m, 54 s            85.23 K                                
26470 (88.23%)      49.355 ms            2 m, 54 s            85.19 K                                
---------------------------------------------------------------------------------------------------- 
Viewer at: https://viewer.nerf.studio/versions/23-03-9-0/?websocket_url=ws://localhost:7007          
Step (% Done)       Train Iter (time)    ETA (time)           Train Rays / Sec                       
-------------------------------

# Render output

In [ ]:
# @title 🗃️ Setup and start viewer
%cd /content

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt

with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)
from IPython import display
display.IFrame(src=url, height=800, width="100%")

In [ ]:
# @title 👀 Visualize training result
!ns-viewer --load-config $root/model/nerfacto/$experiment_name/config.yml

[05:09:09] Skipping 0 files in dataset split train.                                         ]8;id=521940;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=754156;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#165\165]8;;\
           Skipping 0 files in dataset split test.                                          ]8;id=822183;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=309376;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#165\165]8;;\
Setting up training dataset...
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this

In [ ]:
# @title 🎨 Render output
%%time
import os
import json
import subprocess
from datetime import datetime
from google.colab import files

config_source = 'predefined' #@param ['predefined', 'upload']
predefined_config = "low-res" #@param ['low-res', 'high-res']
output_type = "video"  #@param ['images', 'video']

# Go into model folder
model_folder = f'{root}/model/nerfacto/{experiment_name}'
%cd $model_folder

# Remove existing camera_path.json first
!rm -rf camera_path.json

# Update camera_path.json
if config_source == 'predefined':
  camera_paths_folder = '/content/predefined_camera_paths'
  if predefined_config == 'high-res':
    !cp $camera_paths_folder/linear_fps48_highres.json ./camera_path.json
  else:
    !cp $camera_paths_folder/linear_fps48_lowres.json ./camera_path.json
else:
  uploaded = files.upload()
  uploadedFileName = list(uploaded.keys())[0]
  os.rename(uploadedFileName, 'camera_path.json')

# Render output
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
output_video_path = f'renders/output-{timestamp}.mp4'
output_images_folder = f'renders/output-{timestamp}/'

if output_type == 'images':
  print(f'Will render images to {output_images_folder}')
  !ns-render \
    --load-config config.yml \
    --traj filename \
    --camera-path-filename camera_path.json \
    --output-path $output_images_folder \
    --output-format images
else:
  print(f'Will render video to {output_video_path}')
  !ns-render \
    --load-config config.yml \
    --traj filename \
    --camera-path-filename camera_path.json \
    --output-path $output_video_path

/content/gdrive/MyDrive/time-machine/nerfstudio/looking-glass/model/nerfacto/df-4
Will render video to renders/output-20230313-051221.mp4
[05:12:30] Skipping 0 files in dataset split train.                                         ]8;id=297973;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=176076;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#165\165]8;;\
           Skipping 0 files in dataset split test.                                          ]8;id=646209;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=181833;file:///usr/local/lib/python3.9/dist-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#165\165]8;;\
Loading latest checkpoint from load_dir
✅ Done loading checkpoint from 
/content/gdrive/MyDrive/time-machine/nerfstudio/looking-